# EDA
This file will take some time loking at the data and trying to figure out what can be useful. There are quite a few tables so it will be helpful to get familiar with exactly what information is available.  
I use a custom class I defined called DataLoader ot deal with some of the tedium of connecting to the database and making common queries.

In [1]:
import psycopg2
import pandas as pd
import statsmodels as sm

from DataLoader import DataLoader

In [2]:
dl = DataLoader.DataLoader('config.json') 

There are quite a few table sin the database and a lot of them don't contain much information. I grouped them into a couple categories based on how useful they would be for predicting match outcomes from ingame data, the main purpose of this project.

In [3]:
dl.get_table(dl.tablename_query, index=None)

,table_name
0,maps
1,users
2,files
3,objects
4,civilization_bonuses
5,alembic_version
6,player_colors
7,object_instance_states
8,tournaments
9,civilizations


## Miscellaneous Tables
These tables contain information that might be useful to someone but not really me, at least for now. I might use some more of the information about series and tournaments for other things but for now I will leave them aside.  
users, files, players, people, hc, events, rounds, series, and series_metadata

## Reference tables
There are quite a few tables that are present just to convert between numeric identifiers and other datatypes, primarily strings, so I won't spend much time with those. The tables maps, civbonus, color, civ, mapsize, eventmap, datasets, gametype, tech, difficulties, mapreveal, speed, startres, startage, victory_conditions, terrain, versions, actions, resources, formation_types, objects, tournaments and ladder all fall into this category.


In [4]:
dl.color

,name,hex
id,,
0,Blue,#0000FF
1,Red,#FF0000
2,Green,#00FF00
3,Yellow,#FFFF00
5,Purple,#FF00FF
6,Gray,#2F2F2F
7,Orange,#FF8201
4,Teal,#00FFFF


In [17]:
dl.datasets

,name,short
id,,
1,Wololo Kingdoms,wk
0,The Conquerors,aoc
100,Definitive Edition,de
200,Age of Kings,aok
7,Realms,realms
2,Portuguese Civilization Mod III,pcm


In [5]:
dl.civbonus.head()

,civilization_id,dataset_id,type,description
id,,,,
1,1,1,civ,Town Centers cost -50% wood in Castle Age
2,1,1,civ,Foot Archers (except Skirmishers) have +1/+2 r...
3,1,1,civ,Shepherds work 25% faster
4,1,1,team,Archery Ranges work 20% faster
5,2,1,civ,Foragers work 25% faster


## Match Data
These are the tables that I will primarily be working with. The reference tables will be useful for translating some things into something more understandable than the numbers while I am getting more familiar with the way that they are encoded but this is where I will spend the rest of my time in this file.

In [6]:
dl.get_table('SELECT COUNT(*) FROM matches')

,count
0,2153310


In [21]:
dl.get_table('SELECT COUNT(DISTINCT match_id) FROM timeseries')

,count
0,14223


The number of matches which have basic info is much higher than those with information of things happening in game so I will subset the tables to just those. 

In [45]:
match_columns = 'id,dataset_id,ladder_id,rated,map_name,played,duration,completed,postgame,map_reveal_choice_id,speed_id,diplomacy_type,starting_resourceS_id,starting_age_id,victory_condition_id,multiqueue,map_tiles,version_id,objects,winning_team_id'
df = dl.get_table('SELECT {0} FROM matches WHERE id IN (SELECT DISTINCT match_id FROM timeseries)'.format(match_columns))

In [47]:
df.ladder_id.value_counts()

131.0    11694
132.0     1861
171.0      200
112.0      115
3.0          1
Name: ladder_id, dtype: int64

In [56]:
df.map_name.value_counts().head(30)

Arabia                    4685
HC2_Arabia                3974
ECL_Arabia                1246
KotD2 - Arabia             838
Arena                      641
Nomad                      543
Green Arabia               166
HC2_Arabia_v2              130
KotD2_Arabia_v2            125
KotD2 - Arabia - 2         101
NAC_Arabia                  87
!! G. A voobly              81
AAN_HC2_Arabia              66
! G.A Voobly                62
NoCut+Allied_Vision_BF      56
MegaRandom (WCL 7)          56
ECL_Arabia_                 45
HC2_Mappack                 39
HC_Arabia                   38
!KotD2 - Arabia             36
Arena [TiTaNiC] v4          34
-  Arabia verde -           33
KotD 19 - Arabia            27
: HC2_Arabia                26
HC2_Cup                     25
ECL_Bedouins                25
BoA_Arabia                  24
Gold Rush                   22
ClownCup - Arena            20
Dry Arabia                  20
Name: map_name, dtype: int64

Arabia is the most standard and while there are differences across versions it should be reasonable to consider each of the scripts that generate Arabia as the same. 

In [59]:
df = dl.get_table("SELECT {0} FROM matches WHERE id IN (SELECT DISTINCT match_id FROM timeseries) AND map_name SIMILAR TO '%(Arabia|G\.? ?A)%'".format(match_columns))

In [60]:
df.shape

(11963, 20)

In [62]:
df.map_name.value_counts()

Arabia                        4685
HC2_Arabia                    3974
ECL_Arabia                    1246
KotD2 - Arabia                 838
Green Arabia                   166
HC2_Arabia_v2                  130
KotD2_Arabia_v2                125
KotD2 - Arabia - 2             101
NAC_Arabia                      87
!! G. A voobly                  81
AAN_HC2_Arabia                  66
! G.A Voobly                    62
ECL_Arabia_                     45
HC_Arabia                       38
!KotD2 - Arabia                 36
-  Arabia verde -               33
KotD 19 - Arabia                27
: HC2_Arabia                    26
BoA_Arabia                      24
ZR@MSC_Bog_Arabia               20
Dry Arabia                      20
AV_Arabia                       16
NAC2_Arabia                     13
HC2_Arabia - Allied Vision      12
- Dry Arabia                    11
- Green Arabia -                 7
Dry  Arabia                      7
HC2_Arabia_No_Wall               7
G.A VOOBLY          